# 对象详解
dv文件可分为四个层次：工程、音轨、区段、音符。dvfile模块为它们分别建立了类，并提供一些函数操作。


In [1]:
import dvfile
d=dvfile.opendv("zhiyaopingfan.dv")

## 几个基本概念
tick：是dv文件中的时间单位，480tick为一个四分音符，与midi标准相同。

## 工程对象 Dvfile
一个Dvfile对象储存一个dv文件。用opendv函数打开dv文件后返回的便是Dvfile对象。

注意Dvfile与dvfile的区别：全小写的dvfile是模块名，首字母大写的Dvfile是dv工程对象。

Dvfile包含以下属性：
- track：音轨列表，每一项是一个Dvtrack对象

In [2]:
d.track

- tempo：曲速标记列表。其中每个曲速标记是元组，元组第一项为曲速标记所处位置（int，单位tick），第二项为该曲速标记后的曲速（float）

In [3]:
d.tempo   #位置为0tick,曲速为每分钟174拍

[(0, 174.0)]

大部分歌曲为非变速曲，全曲只有一个曲速标记。对于这些歌曲，可以用以下方式获取曲速值

In [4]:
d.tempo[0][1]

174.0

- beats：节拍标记列表。其中每个节拍标记是元组，元组第一项为节拍标记所处小节，第二项为分子（每小节拍数，int），第三项为分母（x分音符为1拍，int，只能为1,2,4,8,16,32）

In [5]:
d.beats   #位置为-3小节，节拍为3/4

[(-3, 3, 4)]

- inst：伴奏音轨列表，每一项是一个Dvinst对象

## 音轨对象 Dvtrack
Dvtrack表示一个dv合成音轨。

In [6]:
tr=d.track[0]
type(tr)

dvfile.Dvtrack

Dvtrack包含以下属性：
- segment：区段列表，每一项是一个Dvsegment对象

In [7]:
tr.segment

- name：音轨名称，str
- volume：音量，int，取值范围[0,100]
- balance：左右声道平衡，int，取值范围[-50,50]
- mute：静音，bool
- solo：独奏，bool

In [8]:
print(tr.name,tr.volume,tr.balance,tr.mute,tr.solo)

歌声-音轨 100 0 True False


## 区段对象 Dvsegment
Dvsegment表示一个dv区段

In [9]:
seg=tr.segment[0]
type(seg)

dvfile.Dvsegment

Dvsegment包含以下属性：
- note：音符列表，每一项是一个Dvnote对象

In [10]:
seg.note

- start：起点tick，从-3小节算起，int
- length：长度tick，int
- name：区段名，str
- singer：音源名，str

In [11]:
print(seg.start,seg.length,seg.name,seg.singer)

5760 111360 新区段 飞梦_CHN_Normal_Build006


    以下是区段线性参数，将会在以后详细介绍
- vol：音量Volume，取值范围[0,256]，numpy.array([[x,y]])
- pit：音调Pitch，以音分为单位，转换成midi标准的100倍，-1表示按默认音调，numpy.array([[x,y]])
- bre：气声Breathness，取值范围[0,256]，numpy.array([[x,y]])
- gen：声线（性别）Gender，取值范围[0,256]，numpy.array([[x,y]])

## 音符对象 Dvnote
Dvnote表示一个dv音符

In [12]:
n=seg.note[0]

Dvnote包含以下属性

    重要属性
- start:起点tick，相对于区段开头的位置，int

In [13]:
n.start

7200

- length:长度tick，int

In [14]:
n.length

480

- notenum:音高，与midi相同，即C4为60，音高越高，数值越大，int

In [15]:
n.notenum

58

- hanzi:歌词汉字，str

In [16]:
n.hanzi

'也'

- pinyin:歌词拼音，str

In [17]:
n.pinyin

'ye'

    其他属性
- benlen:弯曲长度，int
- bendep:弯曲深度，int
- porhead:头部滑音长度，int
- portail:尾部滑音长度，int
- timbre：采样音阶，-1表示跟随音高，0表示T1，1表示T2，以此类推，int
- viblen:颤音长度
- vibamp:颤音幅度曲线，numpy.array([[x,y]])
- vibfre:颤音速度曲线，numpy.array([[x,y]])
- vibp:渲染出的颤音音高曲线，numpy.array([[x,y]])，单位：（毫秒，音分），每10ms采样一次
- crolrc:交叉拼音，str
- crotim:交叉音阶，int

## 例子
以下程序将工程文件中的所有音符向上移动一个半音

In [18]:
import dvfile
d=dvfile.opendv("zhiyaopingfan.dv")
for tr in d.track:#遍历音轨
    for seg in tr.segment:#遍历区段
        for n in seg.note:#遍历音符
            n.notenum+=1
d.save("zhiyaopingfan3.dv")